In [1]:
%load_ext autoreload
%autoreload 2

import warnings
from pathlib import Path

from leaf_disease.datasets.dataset import LeafImageDataModule, DataAugmentation
from leaf_disease.models import Resnet, Efficientnet, ResnetSSL
from leaf_disease.lit_models.lit_model import LitModel

import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.utilities.warnings import PossibleUserWarning

warnings.filterwarnings("ignore", category=PossibleUserWarning)

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
# model transforms
from torchvision.models import resnet

resnet.ResNet50_Weights.IMAGENET1K_V1.transforms()

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [3]:
resnet.ResNet50_Weights.IMAGENET1K_V2.transforms()

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [5]:
# Albumentations library
image_size = 512

transforms = DataAugmentation(image_size)
transforms

In [7]:
# {
#     "resnet18": {"model": "resnet18", "weights": "ResNet18_Weights.IMAGENET1K_V1"},
#     "resnext50_32x4d_v1": {"model": "resnext50_32x4d", "weights": "ResNeXt50_32X4D_Weights.IMAGENET1K_V1"},
#     "resnext50_32x4d_v2": {"model": "resnext50_32x4d", "weights": "ResNeXt50_32X4D_Weights.IMAGENET1K_V2"}
# }

In [8]:
train_image_path = Path("input")

# Data module
dm = LeafImageDataModule(
    image_path=train_image_path, 
    batch_size=32, 
    num_workers=4,
    transforms=transforms
)

# download resnet18 from the hub
# The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. 
# You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights
# resnext50_32x4d_ssl
model_name = "resnext50_32x4d"
weights = "ResNeXt50_32X4D_Weights.IMAGENET1K_V2"
pytorch_model = Resnet(num_classes=5, model=model_name, weights="DEFAULT")

# model_name = "efficientnet_b0"
# pytorch_model = Efficientnet(num_classes=5, model=model_name, weights="DEFAULT")
# pytorch_model

Using cache found in /Users/cespeleta/.cache/torch/hub/pytorch_vision_main


In [9]:
# Initialize Lightning model.
lit_model = LitModel(pytorch_model=pytorch_model, learning_rate=1e-4, weight_decay=1e-6)

In [10]:
monitor = "val_acc"
mode = "max"

callbacks = [
    EarlyStopping(monitor=monitor, patience=2, verbose=True, mode=mode),
    ModelCheckpoint(filename="best", monitor=monitor, mode=mode, verbose=True, save_last=True),
    LearningRateMonitor(logging_interval="epoch", log_momentum=True)
]

In [11]:
L.seed_everything(123)

trainer = L.Trainer(
    callbacks=callbacks,
    max_epochs=10,
    accelerator="mps",
    devices=1,
    logger=TensorBoardLogger(save_dir="logs/", name=model_name, sub_dir="tf_logs"),
    deterministic=True,
    gradient_clip_val=0.1,  # from Kaggle Notebook
    num_sanity_val_steps=1
)

Global seed set to 123
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(model=lit_model, datamodule=dm)

Missing logger folder: logs/resnext50_32x4d


len(train_image_paths)=17117
len(valid_image_paths)=4280



  | Name          | Type               | Params | In sizes       | Out sizes
----------------------------------------------------------------------------------
0 | pytorch_model | Resnet             | 23.0 M | [1, 3, 32, 32] | [1, 5]   
1 | train_acc     | MulticlassAccuracy | 0      | ?              | ?        
2 | valid_acc     | MulticlassAccuracy | 0      | ?              | ?        
3 | test_acc      | MulticlassAccuracy | 0      | ?              | ?        
----------------------------------------------------------------------------------
23.0 M    Trainable params
0         Non-trainable params
23.0 M    Total params
91.961    Total estimated model params size (MB)


/Users/cespeleta/Projects/cassava-leaf-disease/.venv/lib/python3.11/site-packages/torchmetrics/functional/classification/accuracy.py:65: UserWarning: MPS: no support for int64 reduction ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:144.)
  tp = tp.sum(dim=0 if multidim_average == "global" else 1)


Epoch 0: 100%|██████████| 534/534 [08:11<00:00,  1.09it/s, v_num=0, val_loss=0.473, val_acc=0.839, train_acc=0.782]

...

Epoch 4: 100%|██████████| 534/534 [07:56<00:00,  1.12it/s, v_num=0, val_loss=0.454, val_acc=0.850, train_acc=0.894]

Epoch 4, global step 2670: 'val_acc' was not in top 1


Epoch 5: 100%|██████████| 534/534 [07:56<00:00,  1.12it/s, v_num=0, val_loss=0.467, val_acc=0.850, train_acc=0.909]

Monitored metric val_acc did not improve in the last 2 records. Best score: 0.850. Signaling Trainer to stop.
Epoch 5, global step 3204: 'val_acc' was not in top 1


Epoch 5: 100%|██████████| 534/534 [07:57<00:00,  1.12it/s, v_num=0, val_loss=0.467, val_acc=0.850, train_acc=0.909]


In [13]:
trainer.callbacks[0].best_score

tensor(0.8832, device='mps:0')

# Inference

In [ ]:
dm.setup()
test_dl = dm.predict_dataloader()

In [ ]:
import numpy as np
import torch

# Load checkpoints
chk_path = trainer.callbacks[-1].best_model_path
# chk_path = "logs/run-top3-512/resnext50_32x4d/checkpoints/best.ckpt"
print(chk_path)
chk = torch.load(chk_path)

# Prepare weights dict
# model_weights = {k: v for k, v in chk["state_dict"].items()}
model_weights = {k.replace("model.", ""): v for k, v in chk["state_dict"].items()}
model_weights = {k.replace("pytorch_model.", ""): v for k, v in chk["state_dict"].items()}

# Init model and set eval mode
model = Resnet(model="resnext50_32x4d")
model.load_state_dict(model_weights, strict=False)
model.to("mps")
model.eval()

In [ ]:
import pandas as pd
test_df = pd.read_csv("input/sample_submission.csv")

final_preds = []
with torch.no_grad():
    for xb, _ in test_dl:
        xb = xb.to("mps")  #.unsqueeze(0)
        batch_pred = model(xb)
        final_preds.append(batch_pred.to("cpu").detach())

final_pred_class = np.vstack(final_preds).argmax(axis=1)
test_df.label = final_pred_class
# test_df.to_csv("submission.csv", index=False)
test_df